In [1]:
import pandas as pd

In [2]:
df= pd.read_csv('taac_assistant_taac.csv')

In [3]:
df.head()

,TaskId,User_Search_Term,Ad,Website,Relevance
0,1,wwww ncquickpass com,Nc Quick Pass - Pay Your Bill Online,www.doxo.com/pay/nc-quick-pass,Other
1,2,peloton plano tx,Studio Cycle Comparison - Find The Best Exerci...,www.nordictrack.com/Studio-Cycles/S22i,Other
2,3,antelope canyon,Hotels near Antelope Canyon - 100% Real Custom...,www.booking.com/Antelope-Canyon/Hotels,Other
3,4,get vaccine after covid,Janssen COVID-19 Vaccine - Authorized For Emer...,www.janssencovid19vaccine.com,Other
4,5,ahs.com/my-accountlogin,Find First american home warranty login - Chec...,www.searchandshopping.org/Your Search/Results,Other


In [4]:
#Converting the Relevance to numerical values to enable analysis
#Convert Relevance to numerical
mapping = {
    'Good' : 1,
    'Other' : 0,
}
df['Relevance'] = df['Relevance'].replace(mapping)
df.head()

,TaskId,User_Search_Term,Ad,Website,Relevance
0,1,wwww ncquickpass com,Nc Quick Pass - Pay Your Bill Online,www.doxo.com/pay/nc-quick-pass,0
1,2,peloton plano tx,Studio Cycle Comparison - Find The Best Exerci...,www.nordictrack.com/Studio-Cycles/S22i,0
2,3,antelope canyon,Hotels near Antelope Canyon - 100% Real Custom...,www.booking.com/Antelope-Canyon/Hotels,0
3,4,get vaccine after covid,Janssen COVID-19 Vaccine - Authorized For Emer...,www.janssencovid19vaccine.com,0
4,5,ahs.com/my-accountlogin,Find First american home warranty login - Chec...,www.searchandshopping.org/Your Search/Results,0


In [5]:
## Get the Independent Features

X=df.drop('Relevance',axis=1)

In [6]:
## Get the Dependent features
y=df['Relevance']

In [7]:
y.value_counts()

1    8
0    7
Name: Relevance, dtype: int64

In [8]:
X.shape

(15, 4)

In [9]:
y.shape

(15,)

In [10]:
import tensorflow as tf

In [11]:
tf.__version__

'2.12.0'

In [12]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [13]:
### Vocabulary size
voc_size=5000

Onehot Representation

In [14]:
messages=X.copy()

In [15]:
messages['User_Search_Term'][1]

'peloton plano tx'

In [16]:
messages['Ad'][1]

'Studio Cycle Comparison - Find The Best Exercise Bike - NordicTrack Official Site'

In [17]:
messages.reset_index(inplace=True)

In [18]:
import nltk
import re
from nltk.corpus import stopwords

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [20]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['User_Search_Term'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [21]:
corpus

['wwww ncquickpass com',
 'peloton plano tx',
 'antelop canyon',
 'get vaccin covid',
 'ah com accountlogin',
 'nike',
 'cfl fixtur',
 'nationwid pet insur',
 'nike',
 'use car',
 'car rental lubbock tx',
 'augusta tech adn',
 'hampton inn guntersvil al',
 'white strip',
 'florist laguna beach']

In [22]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[691, 3454, 3488],
 [166, 4790, 1165],
 [3366, 3585],
 [2574, 4033, 734],
 [4605, 3488, 4409],
 [886],
 [1691, 3489],
 [1918, 3797, 4070],
 [886],
 [2606, 3874],
 [3874, 4264, 1050, 1165],
 [199, 1752, 2798],
 [2422, 930, 3500, 1086],
 [3385, 4137],
 [3092, 4299, 761]]

Embedding Representation

In [26]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0  691 3454 3488]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0  166 4790 1165]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0 3366 3585]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0 2574 4033  734]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0 4605 3488 4409]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0  886]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0 1691 3489]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0 1918 3797 4070]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0  886]
 [   0    0    0    0    0    0    0    0    0    0    

In [27]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,  691, 3454, 3488], dtype=int32)

In [28]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [30]:
len(embedded_docs),y.shape

(15, (15,))

In [31]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [32]:
X_final.shape,y_final.shape

((15, 20), (15,))

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

Model Training

In [34]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=8,batch_size=16)

Epoch 1/8
1/1 [==============================] - 6s 6s/step - loss: 0.6920 - accuracy: 0.7000 - val_loss: 0.6938 - val_accuracy: 0.4000
Epoch 2/8
1/1 [==============================] - 0s 78ms/step - loss: 0.6918 - accuracy: 0.5000 - val_loss: 0.6939 - val_accuracy: 0.4000
Epoch 3/8
1/1 [==============================] - 0s 65ms/step - loss: 0.6934 - accuracy: 0.3000 - val_loss: 0.6937 - val_accuracy: 0.4000
Epoch 4/8
1/1 [==============================] - 0s 74ms/step - loss: 0.6944 - accuracy: 0.6000 - val_loss: 0.6932 - val_accuracy: 0.4000
Epoch 5/8
1/1 [==============================] - 0s 65ms/step - loss: 0.6927 - accuracy: 0.5000 - val_loss: 0.6925 - val_accuracy: 0.6000
Epoch 6/8
1/1 [==============================] - 0s 66ms/step - loss: 0.6916 - accuracy: 0.6000 - val_loss: 0.6920 - val_accuracy: 0.8000
Epoch 7/8
1/1 [==============================] - 0s 62ms/step - loss: 0.6888 - accuracy: 0.8000 - val_loss: 0.6917 - val_accuracy: 0.8000
Epoch 8/8
1/1 [=====================

Performance Metrics And Accuracy

In [35]:
y_pred1 = np.argmax(model1.predict(X_test),axis=1)

1/1 [==============================] - 1s 753ms/step


In [36]:
y_pred1

array([0, 0, 0, 0, 0])

In [37]:
from sklearn.metrics import confusion_matrix

In [38]:
confusion_matrix(y_test,y_pred1)

array([[2, 0],
       [3, 0]])

In [39]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.4

In [40]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.40      1.00      0.57         2
           1       0.00      0.00      0.00         3

    accuracy                           0.40         5
   macro avg       0.20      0.50      0.29         5
weighted avg       0.16      0.40      0.23         5



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [41]:
predict_relevance = ["enterprise rent-a-car", "Enterprise Rent-A-Car - When You're Ready, Were Ready - Enterprise Official Site",
               "www.enterprise.com"]
def predict_rel(predict_relevance):
  onehot_reprr=[one_hot(words,voc_size)for words in predict_relevance]
  padded = pad_sequences(onehot_reprr,
                         maxlen = sent_length,
                         padding = 'pre')

  return(model1.predict(padded))
predict_rel(predict_relevance)

1/1 [==============================] - 0s 24ms/step


array([[0.50259554],
       [0.50488526],
       [0.49715364]], dtype=float32)

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

In [ ]:
model1.predict([["enterprise rent-a-car","Enterprise Rent-A-Car - When You're Ready, Were Ready - Enterprise Official Site","www.enterprise.com"]])

UnimplementedError: ignored